# Practical exercise

## Machine Learning in der Medizin

*authors: Asan Agibetov, Geord Dorffner*

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Standard-library imports
import math
import random


# Third-party imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Normal
import click
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(0)

## Neural Network definition

In [4]:
class Approxnet(nn.Module):
    def __init__(self, input_dim, hids_dims, out_dim, activation="relu"):
        super(Approxnet, self).__init__()

        self.activation = activation
        self.activation_fn = F.relu if activation == "relu" else F.sigmoid

        self.input = nn.Linear(input_dim, hids_dims[0])
        
        self.hids = nn.ModuleList(
                [nn.Linear(h1, h2) for h1, h2 in zip(hids_dims[:-1], hids_dims[1:])])
        self.output = nn.Linear(hids_dims[-1], out_dim)
        
        self.init_weights()
        
    def init_weights(self):
        self.input.weight.data.normal_(0, 1./math.sqrt(self.input.weight.size(1)))
        self.input.bias.data.zero_()
        
        for hid in self.hids:
            hid.weight.data.normal_(0, 1./math.sqrt(hid.weight.size(1)))
            hid.bias.data.zero_()
        
        self.output.weight.data.normal_(0, 1./math.sqrt(self.output.weight.size(1)))
        self.output.bias.data.zero_()
        

    def forward(self, x, hidden=False):
        """Output activations of the last hidden neuron layer"""
        output = self.activation_fn(self.input(x))

        for hid in self.hids:
            output = self.activation_fn(hid(output))

        if hidden:
            return output

        # if self.activation == "sigmoid":
        #    return self.activation_fn(self.output(output))

        return self.output(output)


    def max_weight_neurons(self):
        """Sort neuron activity by the weights"""
        pass


    def min_weight_neurons(self):
        """Sort neuron activity by the weights"""
        pass

## Data generation

In [5]:
def generate_data(n_samples):
    # Generate 100 random points, the values we want to approximate
    EPS = 1e-5
    x = torch.linspace(-2*math.pi, 2*math.pi, n_samples).view(-1, 1)
    x_generate = torch.from_numpy(np.random.uniform(-2*math.pi, 2*math.pi, (n_samples,))).view(-1, 1).float()
    
    true_function = lambda x: torch.cos(x)**2 + torch.cos(x)**3 + torch.sin(x)**5
    
    y = true_function(x_generate)
    # y = torch.cos(x) + 1.2
    
    # y /= y.max()
    # y += y.min() + EPS
    
    mean_y, std_y = y.mean(), y.std()
    y -= mean_y
    y /= std_y
    
    # x = (x - x.mean())/x.std()

    return x, x_generate, y, true_function

## Training network

In [6]:
def train(model, data, opts):
    """Train a classifier on random points, i.e., a NN that learns to
    reconstruct a function whose values are these random points"""
    x, y = data

    num_epochs = opts["num_epochs"]
    batch_size = opts["batch_size"]
    lr = opts["lr"]
    moment = opts["moment"]

    # optimizer
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=moment)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)

    model.train()

    nsamples = len(x)
    # get indices of random samples for the batch generator
    perm = torch.randperm(nsamples).long()

    num_batches = range(1, math.ceil(nsamples / batch_size))

    pbar = tqdm(enumerate(range(1, num_epochs + 1)), total=num_epochs, leave=True)
    for i, epoch in pbar:
        total_loss = 0.0

        for batch in num_batches:
            inputs = x[perm[batch*batch_size:(batch+1) * batch_size]].view(-1, 1).float()
            targets = y[perm[batch*batch_size:(batch+1) * batch_size]].float()

            inputs, targets = Variable(inputs), Variable(targets)
            
            optimizer.zero_grad()

            output = model(inputs)
            loss = F.mse_loss(output, targets, size_average=False)


            total_loss += loss.data[0]
            
            # compute gradients
            # backpropagate: update weights
            loss.backward()        
            optimizer.step()
            
        scheduler.step(total_loss)

        # log partial results
        pbar.set_description("epoch: {0}, avg loss: {1:.2f}".format(epoch, total_loss/batch))

## Function approximation

In [7]:
def main(activation, hid_dims, num_samples, num_epochs, batch_size, lr, moment, topk):
    # Training settings
    opts = {
        "hid_dims": hid_dims,
        "acitvation": activation,
        "num_epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "moment": moment,
    }

    print("training model with params")
    for key, value in opts.items():
        print("{}: {}".format(key, value))

    # Model, i.e., Neural Network
    input_dim = 1
    output_dim = 1

    model = Approxnet(input_dim, hid_dims, output_dim, activation=activation)
    print(model)

    print("generating {} samples".format(num_samples))
    x, x_generate, y, true_func = generate_data(num_samples)

    try:
        train(model, (x_generate, y), opts)

    except KeyboardInterrupt:
        print("Early stopping")


    model.eval()

    inputs = Variable(x)
    output = model(inputs)
    output_hidden = model.forward(inputs, hidden=True)

    plt.plot(x.squeeze().numpy(), true_func(x).squeeze().numpy(), 'g--', label="true function")
    plt.plot(x.squeeze().numpy(), output.data.squeeze().numpy(), 'r', label="approximation")
    
    print("MSE approximation and true function {}".format(torch.norm(output.data - y)))

    # find neurons that contribute the most
    # weights, hidden_idxs = torch.topk(model.output.weight, topk)
    # weights, hidden_idxs = map(lambda x: x.data.squeeze(), [weights, hidden_idxs])

    # show the function they approximate
    # activations = []
    # for weight, hidden_idx in zip(weights, hidden_idxs):
    #    activation = output_hidden[:, hidden_idx].data.squeeze() * weight
    #    activations.append(activation)
        
    # final_activations = activations[0]
    # for i in range(1, len(activations)):
    #    final_activations += activations[i]

    # plt.plot(x.squeeze().numpy(), final_activations.numpy(), 'b',
    #         label="{} most contributing neurons".format(topk))

    plt.legend()
    plt.show()

## Run your script

In [1]:
activation = "sigmoid"
hid_dims = [20]
num_samples = 8000
num_epochs = 100
batch_size = 500
lr = 1e-5
moment = 0.9
topk = 10

In [2]:
main(activation, hid_dims, num_samples, num_epochs, batch_size, lr, moment, topk)

NameError: name 'main' is not defined